## resizing and saving images

In [1]:
# def resize_im(
#     image_name, new_folder_name="small_nearest", size=(224, 224), resample=Image.NEAREST
# ):
#     image = Image.open(PATH + "train_images/" + image_name)
#     image = image.resize(size, resample=resample)
#     if not os.path.isdir(PATH + new_folder_name + "/"):
#         os.mkdir(PATH + new_folder_name + "/")
#     image.save(PATH  + new_folder_name + "/" + image_name)
# labels['image'].apply(resize_im)

## weights for multilabels

In [ ]:
# labels["multi_label"] = labels["labels"].astype("category").cat.codes
# dict_weights = (
#     1
#     / labels["multi_label"].value_counts()
#     / np.sum(1 / labels["multi_label"].value_counts())
# ).to_dict()
# def weight_fill(x):
#     return dict_weights[x]
# labels["weights"] = labels["multi_label"].apply(weight_fill)

In [ ]:
# class MyImageDataGenerator(ImageDataGenerator):
#     def __init__(self, std_normalization=True):
#         self.std_normalization = std_normalization

#     if self.std_normalization:
#         x = x.astype(np.float32)
#         x = x - np.mean(x, axis=(0, 1))
#         x = x / np.std(x, axis=(0, 1))

#     else:
#         warnings.warn("Ничего не получилось")

## find duplicates

In [ ]:
# import os
# from PIL import Image, ImageStat

# image_folder = os.path.join(PATH, "small_nearest")
# image_files = labels['image'].tolist()

mean pix for each layer

In [ ]:
# duplicate_files_3 = pd.DataFrame()
# for file in tqdm(image_files):
#     image = Image.open(os.path.join(image_folder, file))
#     pix_mean = ImageStat.Stat(image).mean
#     duplicate_files_3.loc[file, 'pix_mean1'] = pix_mean[0]
#     duplicate_files_3.loc[file, 'pix_mean2'] = pix_mean[1]
#     duplicate_files_3.loc[file, 'pix_mean3'] = pix_mean[2]


In [ ]:
# duplicate_files = duplicate_files.reset_index(drop=False)
# duplicate_files.columns = ['image', 'pix_mean1', 'pix_mean2', 'pix_mean3']

#### np.round or without

In [ ]:
# duplicates_array = np.round(duplicate_files[['pix_mean1', 'pix_mean2', 'pix_mean3']].values)
# duplicates_array[0]

In [ ]:
# list_duplicates = []
# for i in tqdm(range(len(duplicates_array))):
#     for j in range(len(duplicates_array)):
#         val1 = duplicates_array[i]
#         val2 = duplicates_array[j]
#         if i != j:
#             if all(val1 == val2):
#                 list_duplicates.append([duplicate_files.loc[i, 'image'], duplicate_files.loc[j, 'image']])

In [ ]:
# df_duplicates = pd.DataFrame(list_duplicates)


In [ ]:
# for im1, im2 in df_duplicates.values:
#     plt.figure(figsize=(10,3))
#     plt.subplot(1,2,1)
#     plt.title(labels[labels['image'] == im1]['labels'].tolist())
#     plt.imshow(Image.open(image_folder+'/'+im1))
#     plt.subplot(1,2,2)
#     plt.title(labels[labels['image'] == im2]['labels'].tolist())
#     plt.imshow(Image.open(image_folder+'/'+im2))
#     plt.show();
# plt.show();

## duplicates via hash

In [ ]:
# import imagehash
# funcs = [
#         imagehash.average_hash,
#         imagehash.phash,
#         imagehash.dhash,
#         imagehash.whash,
#     ]

# image_ids = []
# hashes = []

# for img_id in tqdm(labels['image']):
#     image = Image.open(image_folder+'/'+img_id)
#     image_ids.append(img_id)
#     hashes.append(np.array([f(image).hash for f in funcs]).reshape(256))

In [ ]:
# hashes_all = np.array(hashes)
# hashes_all = torch.Tensor(hashes_all.astype(int))

# %time sims = np.array([(hashes_all[i] == hashes_all).sum(dim=1).numpy()/256 for i in range(hashes_all.shape[0])])

In [ ]:
# indices1 = np.where(sims > 0.9)
# indices2 = np.where(indices1[0] != indices1[1])
# image_ids1 = [image_ids[i] for i in indices1[0][indices2]]
# image_ids2 = [image_ids[i] for i in indices1[1][indices2]]

# dups = [list(sorted([image_ids1,image_ids2])) for image_ids1, image_ids2 in zip(image_ids1, image_ids2)]
# print('found %d duplicates' % len(dups))
# for row in dups:
#     print(','.join(row))

## predict_test

In [ ]:
# def predict_test(threshold = True):
#     sample_submission_copy = sample_submission.copy()

#     if threshold:
#         prediction_valid_results = model.predict(valid)
#         prediction_valid = pd.DataFrame(prediction_valid_results, columns=feature_columns).join(pd.DataFrame(valid._targets, columns=feature_columns), rsuffix='_t')
#         best_thresholds = pd.DataFrame()
#         for col in feature_columns:
#             f1_init = 0
#             for th in np.linspace(0.01, 1, 500):
#                 f1 = metrics.f1_score(prediction_valid[col +'_t'], prediction_valid[col]>th)
#                 if f1>f1_init:
#                     f1_init = f1
#                     best_thresholds.loc['threshold', col] = th
#                     best_thresholds.loc['f1', col] = f1
#         test_prediction = pd.DataFrame(model.predict(test), columns=feature_columns, index = sample_submission.index)
#         for col in feature_columns:
#             sample_submission_copy[col] = test_prediction[col]>best_thresholds.loc['threshold', col]
#             sample_submission_copy[col] = sample_submission_copy[col].replace({True: col, False: ""})
#         sample_submission_copy['labels'] = sample_submission_copy[feature_columns].apply(" ".join, axis=1).str.split().str.join(sep=" ")
#         return sample_submission_copy
# #     , best_thresholds, prediction_valid
#     else:
#         test_prediction = pd.DataFrame(model.predict(test), columns=feature_columns, index = sample_submission.index)
#         for col in feature_columns:
#             sample_submission_copy[col] = test_prediction[col]>0.5
#             sample_submission_copy[col] = sample_submission_copy[col].replace({True: col, False: ""})
#         sample_submission_copy['labels'] = sample_submission_copy[feature_columns].apply(" ".join, axis=1).str.split().str.join(sep=" ")
#         return sample_submission_copy

# sample_submission_copy = predict_test(threshold=True)
# sample_submission_copy[['image', 'labels']].to_csv("submission.csv",index=False)
# sample_submission_copy[['image', 'labels']]

In [ ]:
# pr = model.predict(test)
# sample_submission_copy = sample_submission.copy()
# for i in range(len(pr)):
#     sample_submission_copy.loc[i,'labels'] = ' '.join(list(map(lambda x,y: x*y, [pr>0.5][0][i],feature_columns)))
# sample_submission_copy.to_csv("submission.csv",index=False)
# sample_submission_copy

## UPSAMPLING

In [ ]:
# train_df, valid_df = train_test_split(
#     labels, train_size=0.8, random_state=SEED, stratify=labels["labels"]
# )


# for col in feature_columns:
#     print(col, train_df[train_df[col] == 1][col].sum())



# powdery_mildew = (
#     train_df[train_df["powdery_mildew"] == 1]
#     .sample(
#         n=(3000 - train_df[train_df["powdery_mildew"] == 1].shape[0]),
#         replace=True,
#         random_state=SEED,
#     )
# )
# rust = (
#     train_df[train_df["rust"] == 1]
#     .sample(
#         n=(3000 - train_df[train_df["rust"] == 1].shape[0]),
#         replace=True,
#         random_state=SEED,
#     )
# )
# complex_df = (
#     train_df[train_df["complex"] == 1]
#     .sample(
#         n=(3000 - train_df[train_df["complex"] == 1].shape[0]),
#         replace=True,
#         random_state=SEED,
#     )
# )
# train_df_new = pd.concat(
#     [
#         train_df,
#         powdery_mildew,
#         rust,
#         complex_df,
#     ],axis=0,ignore_index=True, 
# )


# train_df_new.shape
# train_df.shape

## evaluate predictions

In [ ]:
def get_prediction(predict_train =False, predict_valid=False, all_img = False):
    if all_img:
        all_img = ImageDataGenerator(rescale=1.0 / 255).flow_from_dataframe(
        dataframe=labels,
        directory=PATH + "train_images/small_bicubic/",
        x_col="image",
        y_col=feature_columns,
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode="raw",
        seed=SEED,
        interpolation = 'bicubic',
        shuffle=False,
    )
        prediction_all = model.predict(all_img)
        prediction = pd.DataFrame(prediction_all, columns=feature_columns).join(pd.DataFrame(all_img._targets, columns=feature_columns), rsuffix='_true', lsuffix = '_pred')
        prediction.index = all_img.filenames
    elif predict_valid:       
        prediction_valid = model.predict(valid)
        prediction = pd.DataFrame(prediction_valid, columns=feature_columns).join(pd.DataFrame(valid._targets, columns=feature_columns), rsuffix='_true', lsuffix = '_pred')
        prediction.index = valid.filenames
    elif predict_train:
        train = train_datagen.flow_from_dataframe(
            dataframe=labels,
            directory=PATH + "train_images/small_bicubic/",
            x_col="image",
            y_col=feature_columns,
            target_size=IMAGE_SIZE,
            batch_size=BATCH_SIZE,
            class_mode="raw",
            subset="training",
            seed=SEED,
            interpolation = 'bicubic',
            shuffle=False
        )
        prediction_train = model.predict(train)
        prediction = pd.DataFrame(prediction_train, columns=feature_columns).join(pd.DataFrame(train._targets, columns=feature_columns), rsuffix='_true', lsuffix = '_pred')
        prediction.index = train.filenames
    return prediction

In [ ]:
def get_metrics(prediction_df, show=True):
    metrics_df = pd.DataFrame()
    for col in feature_columns:
        pres = metrics.precision_score(prediction_df[col+'_true'], prediction_df[col+'_pred']>0.5)
        rec = metrics.recall_score(prediction_df[col+'_true'], prediction_df[col+'_pred']>0.5)
        f1 = metrics.f1_score(prediction_df[col+'_true'], prediction_df[col+'_pred']>0.5)
        metrics_df.loc['precision', col] = pres
        metrics_df.loc['recall', col] = rec
        metrics_df.loc['f1_score', col] = f1
    if show:
        print(metrics_df)
    return metrics_df

In [ ]:
def wrong_prediction(prediction_df, treshold = 0.5):
    list_wrong_prediction = []
    prediction = prediction_df.copy()
    for col in feature_columns:
        prediction[col+'_pred'] = prediction[col+'_pred'] > treshold
        prediction[col+'_pred'] = prediction[col+'_pred'].replace({True: 1, False: 0})
        ids = prediction[prediction[col+'_true'] != prediction[col+'_pred']].index.tolist()
        list_wrong_prediction.extend(ids)
    list_wrong_prediction = set(list_wrong_prediction)
    wrong_prediction = prediction.loc[list_wrong_prediction]
    for img in wrong_prediction.index.tolist():
        pred = wrong_prediction.loc[img][:6].values
        real = wrong_prediction.loc[img][6:12].values
        pred_names = ' '.join(list(map(lambda x,y: x*y, pred,feature_columns)))
        pred_names = ' '.join(pred_names.split())
        real_names = ' '.join(list(map(lambda x,y: x*y, real,feature_columns)))
        real_names = ' '.join(real_names.split())
        wrong_prediction.loc[img, 'pred_labels'] = pred_names
        wrong_prediction.loc[img, 'real_labels'] = real_names
    return prediction, wrong_prediction

In [ ]:
def plot_wrongs(df, num_im = 20):
    plt.figure(figsize=(15,(num_im//4)*5))
    for i, img in enumerate(df.sample(num_im).index.tolist()):
        image = Image.open('/kaggle/input/plant-small/small_nearest/'+img)
        plt.subplot(num_im//4, 4, i+1)
        plt.title('predicted: '+df.loc[img,'pred_labels']+ '\nreal: '+df.loc[img,'real_labels'])
        plt.imshow(image)
        plt.xticks([])
        plt.yticks([])
    plt.tight_layout()
    plt.show();